## Define the network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # output: torch.Size([1, 16, 6, 6])
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        # flatten
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [18]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [19]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out.shape)

torch.Size([1, 10])


In [20]:
print(out)

tensor([[-0.1406,  0.1502,  0.1581,  0.0190, -0.0339, -0.1123,  0.0403,  0.0103,
          0.0635, -0.0208]], grad_fn=<AddmmBackward>)


In [21]:
net.zero_grad()

In [22]:
out.backward(torch.randn(1, 10))

## loss function

In [23]:
output = net(input)
target = torch.randn(10)
print(target.size())

torch.Size([10])


In [25]:
target = target.view(1, -1)
print(target.size())

torch.Size([1, 10])


In [26]:
criterion = nn.MSELoss()

In [27]:
loss = criterion(output, target)
print(loss)

tensor(1.8933, grad_fn=<MseLossBackward>)


In [28]:
print(loss.grad_fn)

In [29]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## Backprop

In [30]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0027, -0.0202,  0.0381, -0.0234, -0.0067, -0.0105])


## Update the weights

In [31]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [32]:
import torch.optim as optim

In [ ]:
optimzer = optim.SGD(net.parameters(), lr = 0.01)
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer